<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/15_Databricks%7C_Spark_%7C_Pyspark_%7C_Read_Json%7C_Flatten_Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install findspark

In [ ]:
!pip install pyspark

In [ ]:
import findspark
from pyspark.sql.functions import  * # to_date, col, split, char_length, lit, trim
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


df = spark.read.option('multiline', 'true').json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/US_category_id.json")
df.printSchema()
df.show(10)


In [ ]:

#Flatten array of structs and structs
def flatten(df):
  #  compute Complex Fields (Lists and Structs) in Schema
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)

      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):
         df=df.withColumn(col_name,explode_outer(col_name))

      # recompute remaining Complex Fields in Schema
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

In [ ]:
df_flatten = flatten(df)

In [ ]:
df_flatten.show()

In [72]:
import json


data = r'''
{
 "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
 "items": [
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
   "id": "1",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Film & Animation",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA\"",
   "id": "2",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Autos & Vehicles",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg\"",
   "id": "10",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Music",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI\"",
   "id": "15",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Pets & Animals",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM1XVQ9zbGec\"",
   "id": "17",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Sports",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/FJwVpGCVZ1yiJrqZbpqe68Sy_OE\"",
   "id": "18",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Short Movies",
    "assignable": false
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf_DkLN8CouA\"",
   "id": "19",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Travel & Events",
    "assignable": true
   }
  }
 ]
} '''
data = json.loads(data)

data

{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

## PySpark — Flatten JSON/Struct Data Frame dynamically

[JSON/Struct Data Frame dynamically](https://subhamkharwal.medium.com/pyspark-flatten-json-struct-data-frame-dynamically-c2e5d8937dcc)

In [74]:
# Lets create an Example Data Frame to hold JSON data
# Example Data Frame with column having JSON data
_data = [
    ['EMP001', '{"dept" : "account", "fname": "Ramesh", "lname": "Singh", "skills": ["excel", "tally", "word"]}'],
    ['EMP002', '{"dept" : "sales", "fname": "Siv", "lname": "Kumar", "skills": ["biking", "sales"]}'],
    ['EMP003', '{"dept" : "hr", "fname": "MS Raghvan", "skills": ["communication", "soft-skills"], "hobbies" : {"cycling": "expert", "computers":"basic"}}'],
    ['EMP003', '{"dept" : "hr", "fname": "MS Raghvan", "skills": ["communication", "soft-skills"], "hobbies" : {"cycling": "expert", "computers":"basic"}}']
]

# Columns for the data
_cols = ['emp_no', 'raw_data']

# Lets create the raw Data Frame
df_raw = spark.createDataFrame(data = _data, schema = _cols)

# Determine the schema of the JSON payload from the column
json_schema_df = spark.read.json(df_raw.rdd.map(lambda row: row.raw_data))
json_schema = json_schema_df.schema

# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json
df_details = df_raw.withColumn("emp_details", from_json(df_raw["raw_data"], json_schema)).drop("raw_data")
df_details.show(10, False)

df_details.printSchema()

+------+---------------------------------------------------------------------+
|emp_no|emp_details                                                          |
+------+---------------------------------------------------------------------+
|EMP001|{account, Ramesh, NULL, Singh, [excel, tally, word]}                 |
|EMP002|{sales, Siv, NULL, Kumar, [biking, sales]}                           |
|EMP003|{hr, MS Raghvan, {basic, expert}, NULL, [communication, soft-skills]}|
|EMP003|{hr, MS Raghvan, {basic, expert}, NULL, [communication, soft-skills]}|
+------+---------------------------------------------------------------------+

root
 |-- emp_no: string (nullable = true)
 |-- emp_details: struct (nullable = true)
 |    |-- dept: string (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- hobbies: struct (nullable = true)
 |    |    |-- computers: string (nullable = true)
 |    |    |-- cycling: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |    |-- sk

In [71]:
# Python function to flatten the data dynamically
from pyspark.sql import DataFrame
# Create outer method to return the flattened Data Frame
def flatten_json_df(_df: DataFrame) -> DataFrame:
    # List to hold the dynamically generated column names
    flattened_col_list = []

    # Inner method to iterate over Data Frame to generate the column list
    def get_flattened_cols(df: DataFrame, struct_col: str = None) -> None:
        for col in df.columns:
            if df.schema[col].dataType.typeName() != 'struct':
                if struct_col is None:
                    flattened_col_list.append(f"{col} as {col.replace('.','_')}")
                else:
                    t = struct_col + "." + col
                    flattened_col_list.append(f"{t} as {t.replace('.','_')}")
            else:
                chained_col = struct_col +"."+ col if struct_col is not None else col
                get_flattened_cols(df.select(col+".*"), chained_col)

    # Call the inner Method
    get_flattened_cols(_df)

    # Return the flattened Data Frame
    return _df.selectExpr(flattened_col_list)# Generate the flattened DF
flattened_df = flatten_json_df(df_details)
flattened_df.show(10)
# Print Schema
flattened_df.printSchema()

+------+----------------+-----------------+-----------------------------+---------------------------+-----------------+--------------------+
|emp_no|emp_details_dept|emp_details_fname|emp_details_hobbies_computers|emp_details_hobbies_cycling|emp_details_lname|  emp_details_skills|
+------+----------------+-----------------+-----------------------------+---------------------------+-----------------+--------------------+
|EMP001|         account|           Ramesh|                         NULL|                       NULL|            Singh|[excel, tally, word]|
|EMP002|           sales|              Siv|                         NULL|                       NULL|            Kumar|     [biking, sales]|
|EMP003|              hr|       MS Raghvan|                        basic|                     expert|             NULL|[communication, s...|
+------+----------------+-----------------+-----------------------------+---------------------------+-----------------+--------------------+

root
 |-- em

## Flattening deep nested JSON using python and Databricks
[link text](https://sanajitghosh.medium.com/flattening-deep-nested-json-using-python-and-databricks-110d097efa69)

In [80]:
import json
import pandas as pd
from pyspark.sql.functions import explode
from pyspark.sql.functions import col
from pyspark.sql.functions import arrays_zip
from pyspark.sql import functions as F

#load json object
df = spark.read.option("multiline","true").json('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/data.json')

def child_struct(nested_df):
    # Creating python list to store dataframe metadata
    list_schema = [((), nested_df)]
    # Creating empty python list for final flattern columns
    flat_columns = []

    while len(list_schema) > 0:
      # Removing latest or recently added item (dataframe schema) and returning into df variable
          parents, df = list_schema.pop()
          flat_cols = [  col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"   ]

          struct_cols = [  c[0]   for c in df.dtypes if c[1][:6] == "struct"   ]

          flat_columns.extend(flat_cols)
          #Reading  nested columns and appending into stack list
          for i in struct_cols:
                projected_df = df.select(i + ".*")
                list_schema.append((parents + (i,), projected_df))
    return nested_df.select(flat_columns)


from pyspark.sql.functions import *
def master_array(df):
    array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    while len(array_cols)>0:
        for c in array_cols:
            df = df.withColumn(c,explode_outer(c))
        df = child_struct(df)
        array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    return df

df_output = master_array(df)
display(df_output)
df_output.show(50)

DataFrame[id: string, name: string, ppu: double, type: string, topping_id: string, topping_type: string, items_etag: string, items_id: string, items_kind: string, items_snippet_assignable: boolean, items_snippet_channelId: string, items_snippet_title: string, batters_batter_id: string, batters_batter_type: string]

+----+-------------+----+-----+----------+--------------------+--------------------+--------+--------------------+------------------------+-----------------------+-------------------+-----------------+-------------------+
|  id|         name| ppu| type|topping_id|        topping_type|          items_etag|items_id|          items_kind|items_snippet_assignable|items_snippet_channelId|items_snippet_title|batters_batter_id|batters_batter_type|
+----+-------------+----+-----+----------+--------------------+--------------------+--------+--------------------+------------------------+-----------------------+-------------------+-----------------+-------------------+
|0001|         Cake|0.55|donut|      5001|                None|                NULL|    NULL|                NULL|                    NULL|                   NULL|               NULL|             1001|            Regular|
|0001|         Cake|0.55|donut|      5001|                None|                NULL|    NULL|                NUL